# Machine Learning

With the data preparation complete, this step will demonstrate how you can configure a [scikit-learn](http://scikit-learn.org/stable/index.html) or [dask_ml](http://ml.dask.org/) pipeline, but any library, algorithm, or simulator could be used at this stage if it can accept array data. In the next step of the tutorial, [Data Visualization](./06_Data_Visualization.ipynb) you will learn how to visualize the output of this pipeline and diagnose as well as ensure that the inputs to the pipeline have the expected structure.

## Recap: Loading data

In [ ]:
import intake

cat = intake.open_catalog('../catalog.yml')
l5_da = cat.l8.read_chunked()

## Subsetting data

To speed things up for tutorial purposes, we'll use a subset of these data in the following examples. There are many ways to [subset data in xarray](http://xarray.pydata.org/en/stable/indexing.html). Here we select the central third of the data using index selection, which will be around 2k x 2k pixels. 

In [ ]:
nbands, ny, nx = l5_da.shape
bounds = int(ny/3), int(2*ny/3), int(nx/3), int(2*nx/3)
bounds

In [ ]:
l5_da = l5_da[:, bounds[0]:bounds[1], bounds[2]:bounds[3]]
l5_da

In [ ]:
import hvplot.xarray

l5_da.hvplot(kind='image', x='x', y='y', groupby='band', datashade=True)

## Reshaping Data

We'll need to reshape the image to be how dask-ml / scikit-learn expect it: `(n_samples, n_features)` where n_features is the number of bands and n_samples is the total number of pixels in each band. Essentially, we'll be creating a bag of pixels out of each image, where each pixel has multiple features (bands), but the ordering of the pixels is no longer relevant.  In this case we start with an array that is n_bands by n_y by n_x  (7, 2000, 2000) and we need to reshape to an array that is `(n_samples, n_features)` (4e6, 7). We'll first look at using NumPy, then Xarray.

### Numpy

Data can be reshaped at the lowest level using NumPy, by getting the underlying values from the `xarray.DataArray`, and using flatten and transpose to get the right shape. 

In [ ]:
import numpy as np

In [ ]:
arr = l5_da.values
arr.shape

Since we want to flatten along the x and y but not along the band axis, we need to iterate over each band and flatten the data. 

In [ ]:
flattened = np.array([arr[i].flatten() for i in range(arr.shape[0])])
flattened

In [ ]:
flattened.shape

We can reorder the dimensions using `.transpose`

In [ ]:
sample_by_feature = flattened.transpose()
sample_by_feature

In [ ]:
sample_by_feature.shape

Since `numpy.array`s are not labeled data, the semantics of the data are lost over the course of these operations, as the necessary metadata does not exist at the NumPy level.

### Xarray

By using `xarray` methods to flatten the data, we can keep track of the coordinate labels ('x' and 'y') along the way. This means that we have the ability to reshape back to our original array at any time with no information loss.

In [ ]:
flattened_by_band = l5_da.stack(z=('x','y'))
flattened_by_band

We can reorder the dimensions using `Dataset.transpose`:

In [ ]:
sample_by_feature = flattened_by_band.transpose('z', 'band')
sample_by_feature

Now we have the data in the shape that we are looking for: a long array of pixels for each band. As a sanity check we can take a look at the plain `np.array`:

In [ ]:
X = sample_by_feature.values
X

## Other preprocessing

Sometimes values are too big or need more axes, or need to have a affine transformation applied. Here we'll demonstrate doing this in `numpy` or `xarray`.

#### Add an axis:

In [ ]:
np.expand_dims(X, 2).shape

In [ ]:
sample_by_feature.expand_dims(dim='e', axis=2)

#### Rescale:

Rescale (standardize) the data to input to the algorithm since the ML pipeline that we have selected expects input values to be small. 

In [ ]:
(X - X.mean()) / X.std()

In [ ]:
rescaled = (sample_by_feature - sample_by_feature.mean()) / sample_by_feature.std()
rescaled.compute()

**NOTE:** Since the the xarray object is in dask, the actual computation isn't performed until `.compute()`is called. 

In [ ]:
rescaled.values

## ML pipeline
The Machine Learning pipeline shown below is just for the purpose of understanding the shaping/reshaping of the data. In practice you will likely be using a more sophisticated pipeline. Here we will use a version of SpectralClustering from dask_ml that is a scalable equivalent to operations from Scikit-learn that cluster pixels based on similarity (across all bands, which makes it spectral clustering by spectra!).

In [ ]:
from dask_ml.cluster import SpectralClustering
from dask.distributed import Client

In [ ]:
client = Client(processes=False)
client

Now we will compute and persist the rescaled data to feed into the ML pipeline. Notice that X has the shape: `n_samples, n_features` as discussed above. 

In [ ]:
X = client.persist(rescaled)
X.shape

In [ ]:
clf = SpectralClustering(n_clusters=4, random_state=0, gamma=None,
                         kmeans_params={'init_max_iter': 5},
                         persist_embedding=True)

In [ ]:
%time clf.fit(X)

In [ ]:
labels = clf.assign_labels_.labels_.compute()
labels.shape

## Un-flattening

Once the computation is done, the output can used to create a new array with the same structure as the input array. This new output array will have the coordinates needed to be unstacked similarly to how they were stacked. One of the main benefits of using `xarray` for this stacking and unstacking is that allows `xarray` to keep track of the coordinate information for us. 

In [ ]:
template = sample_by_feature[:, 0]
template

**NOTE:** Since the original array is n_samples by n_features (4_000_000, 6) and the output only contains one feature (4_000_000,), the template structure for this data needs to have the shape (n_samples). We achieve this by just taking one of the bands.

In [ ]:
output_array = template.copy(data=labels)
output_array

With this new output array in hand, we can unstack back to the original dimensions:

In [ ]:
unstacked = output_array.unstack()
unstacked

In [ ]:
(unstacked.hvplot(x='x', y='y', datashade=True).relabel('Clustered') + 
 l5_da.sel(band=4).hvplot(x='x', y='y', datashade=True).relabel('Image')).cols(1)

## Geographic plot

The plot above is useful and quick to generate, but it isn't referenced against the underlying geographic coordinates, which is crucial if we want to overlay the data on any other geographic data sources. Adding the coordinate reference system in the hvplot method, ensures that the data is properly positioned in space. This geo-referencing is made very straightforward because of the way `xarray` persists metadata. 

In [ ]:
import cartopy.crs as ccrs

In [ ]:
unstacked.crs

In [ ]:
crs = ccrs.epsg(32611)

In [ ]:
(unstacked.hvplot(x='x', y='y', datashade=True, crs=crs, height=500).relabel('Clustered') +
 l5_da.sel(band=3).hvplot.image(x='x', y='y', datashade=True, crs=crs, height=500).relabel('Image')).cols(1)

### Next:

Now that your analysis is complete, you are ready for some more information about [Data Visualization](./06_Data_Visualization.ipynb) you will learn how to visualize the output of this pipeline and diagnose as well as ensure that the inputs to the pipeline have the expected structure.